In [1]:
import pandas as pd

# Adjust pandas options to see all columns and to read subgroup descriptions fully
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

In [2]:
df_s = pd.read_csv("results/subgroup_ss_mathematics_results8020.csv")
df_s.head(1)
#df_s.columns

,r2,mae,mse,mean_residual,intercept,coef__G1,pval__G1,coef__G2,pval__G2,coef__absences,pval__absences,coef__activities,pval__activities,coef__paid,pval__paid,coef__famsup,pval__famsup,coef__schoolsup,pval__schoolsup,coef__studytime,pval__studytime,model_type,description,cookD,n_train,n_test,global_baseline_r2,global_baseline_mae,global_baseline_mse,global_baseline_mean_residual,mean_baseline_r2,mean_baseline_mae,mean_baseline_mse,mean_baseline_mean_residual,ttest_p,ttest_stat,wilcoxon_p,wilcoxon_stat,ttest_p_mean,ttest_stat_mean,wilcoxon_p_mean,wilcoxon_stat_mean,ttest_p_mean_global,ttest_stat_mean_global,wilcoxon_p_mean_global,wilcoxon_stat_mean_global
0,0.3962,2.612,10.0226,0.1649,-7.86071,-0.237879,0.581558,1.784527,0.00228,0.54957,0.003202,-1.984971,0.185259,-3.420051,0.060566,2.086108,0.130112,1.954133,0.386908,0.399439,0.717674,subgroup,"Fedu in (-0.001, 2.0] ∧ Fjob=='services' ∧ Medu in (-0.001, 2.0]",235.733335,26,10,0.6999,1.3531,4.9811,-0.3325,-0.1498,3.430769,19.086686,1.576923,0.841904,1.061324,0.919922,41.0,0.173281,-0.993202,0.09668,14.0,0.022214,-2.334383,0.041992,10.0


In [3]:
# Assign rank: one per row (since you now have only subgroup_model rows)
df_s['subgroup_rank'] = (df_s.index + 1).astype("object")
df_s.loc[df_s['model_type'] == 'global', 'subgroup_rank'] = "N/A"

# Round columns for better readability
for col, ndigits in [('cookD', 2), ('r2', 3), ('mae', 3), ('mse', 3), ('mean_residual', 3),
                     ('global_baseline_r2', 3), ('global_baseline_mae', 3), ('global_baseline_mse', 3), ('global_baseline_mean_residual', 3),
                     ('mean_baseline_r2', 3), ('mean_baseline_mae', 3), ('mean_baseline_mse', 3), ('mean_baseline_mean_residual', 3)]:
    if col in df_s.columns:
        df_s[col] = df_s[col].round(ndigits)

# Compare r2 between subgroup_model and global baseline for each row
df_s['subgroup_r2_better'] = (df_s['r2'] > df_s['global_baseline_r2']).astype("object")
df_s.loc[df_s['model_type'] == 'global', 'subgroup_r2_better'] = None
df_s['subgroup_better_global'] = (df_s['ttest_p'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_global'] = None
df_s['subgroup_better_mean'] = (df_s['ttest_p_mean'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_mean'] = None
df_s['global_better_mean'] = (df_s['ttest_p_mean_global'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'global_better_mean'] = None

# Add decision column based on the rules
def determine_decision(row):
    sg = row['subgroup_better_global']
    sm = row['subgroup_better_mean']
    gm = row['global_better_mean']
    
    if sg and sm:
        return "True discovery"
    elif sg and not sm:
        return "True discovery, but model not appropriate"
    elif not sg and sm:
        return "Weak discovery"
    elif not sg and not sm and gm:
        return "False discovery"
    else:
        return "Model not appropriate"

df_s['decision'] = df_s.apply(determine_decision, axis=1)

# Specify desired column order
main_cols = [
    'subgroup_rank', 'model_type', 'description', 'decision', 'subgroup_r2_better', 'cookD', 'n_train', 'n_test',
    'r2', 'global_baseline_r2', 'mean_baseline_r2', 
    'mae', 'global_baseline_mae', 'mean_baseline_mae',
    'mse', 'global_baseline_mse', 'mean_baseline_mse',
    'mean_residual', 'global_baseline_mean_residual', 'mean_baseline_mean_residual'
]
rest_cols = [c for c in df_s.columns if c not in main_cols]
new_order = main_cols + rest_cols

# Reorder columns
df_s = df_s[new_order]

/var/folders/0_/xw_ks0g978df8kf5jp1rvf2h0000gn/T/ipykernel_4319/2205722591.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_global'] = None
/var/folders/0_/xw_ks0g978df8kf5jp1rvf2h0000gn/T/ipykernel_4319/2205722591.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_mean'] = None
/var/folders/0_/xw_ks0g978df8kf5jp1rvf2h0000gn/T/ipykernel_4319/2205722591.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bo

In [7]:
df_s[df_s['decision'].isin(['Weak discovery', 'True discovery'])][['subgroup_rank', 'description', 'decision', 'n_test', 'n_train', 'r2', 'global_baseline_r2', 'mean_baseline_r2']]

,subgroup_rank,description,decision,n_test,n_train,r2,global_baseline_r2,mean_baseline_r2
8,9,"Medu in (-0.001, 2.0] ∧ Pstatus=='T' ∧ age in (17.0, 18.0]",Weak discovery,9,27,0.656,0.734,-0.322
9,10,"Medu in (-0.001, 2.0] ∧ age in (17.0, 18.0] ∧ traveltime in (0.999, 2.0]",True discovery,6,26,0.888,0.615,-1.044
12,13,Fjob=='services' ∧ guardian=='father',Weak discovery,8,27,0.824,0.695,-0.056
13,14,"Fjob=='services' ∧ guardian=='father' ∧ traveltime in (0.999, 2.0]",Weak discovery,6,27,0.856,0.712,-0.108
14,15,"Fjob=='services' ∧ failures in (-0.001, 3.0] ∧ guardian=='father'",Weak discovery,8,27,0.824,0.695,-0.056
15,16,Fjob=='services' ∧ Pstatus=='T' ∧ guardian=='father',Weak discovery,8,27,0.824,0.695,-0.056
18,19,"Medu in (-0.001, 2.0] ∧ age in (17.0, 18.0]",Weak discovery,9,28,0.677,0.734,-0.355
19,20,"Medu in (-0.001, 2.0] ∧ age in (17.0, 18.0] ∧ failures in (-0.001, 3.0]",Weak discovery,9,28,0.677,0.734,-0.355
20,21,"Dalc in (0.999, 2.0] ∧ Medu in (-0.001, 2.0] ∧ age in (17.0, 18.0]",Weak discovery,6,28,0.884,0.794,-0.597
21,22,"Fjob=='services' ∧ Medu in (-0.001, 2.0]",Weak discovery,12,36,0.590,0.765,-0.202
